## Numerical Analysis - Fall semester 2024

# Serie 06 - Least squares fitting and numerical differentiation

Importing NumPy and matplotlib, our loyal companions. This time, we will also generate some data for you to use in one of the exercises.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0); np.savetxt("data.txt", np.vstack((np.linspace(0, 10, 20), np.sin(np.linspace(0, 10, 20)) + np.linspace(0, 10, 20) + 0.1 * np.random.randn(20))).T); np.savetxt("big_data.txt", np.vstack((np.linspace(0, 10, 20000), np.sin(np.linspace(0, 10, 20000)) + np.linspace(0, 10, 20000) + 0.1 * np.random.randn(20000))).T)

<hr style="clear:both">

### Least squares approximation of data

Let us consider the data $(x_i,y_i)$, $i=1,\ldots,20$ contained in the file `data.txt` which has been generated for you in the same directory as this notebook is stored in. The data corresponds to the evaluation of the function 

$$
  f(x) = \sin(x) + x, \quad x\in I=[0, 10],
$$

on $n+1=20$ equally distributed nodes (therefore $x_1=0$ and $x_{20}=10$).
The evaluations are perturbed by Gaussian noise as follows:

$$
y_i=f(x_i) + \varepsilon_i, \quad i=1,\ldots,n+1.
$$

Let us suppose that the terms $\varepsilon_i$ are stochastically independent, with zero mean and standard deviation $\sigma=0.1$.

<div class="alert alert-success">
    
**Exercise 1:** Load the data using the function `np.loadtxt('[FILENAME HERE].txt')` and visualize it in a plot. Also add a visualization of the function $f$ in your plot.
</div>

In [ ]:
def f(x):
    ### BEGIN SOLUTION
    return np.sin(x) + x
    ### END SOLUTION

### BEGIN SOLUTION
data = np.loadtxt("data.txt")
x = data[:, 0]
y = data[:, 1]
x_lin = np.linspace(0, 10, 100)
plt.scatter(x, y, label=r"data $(x_i, y_i)$")
plt.plot(x_lin, f(x_lin), color="black", label=r"function $f(x)$")
plt.legend()
plt.show()
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 2:** For $m = 4, 7, 15$, compute and plot the least-squares polynomial $p_{m}^{LS}$ of degree $m$ of the data. Do this in a separate figure for each $m$ where you also plot the function and the data. What do you observe?
</div>

In [ ]:
### BEGIN SOLUTION
m_list = [4, 7, 15]
for m in m_list:
    coeff = np.polyfit(x, y, m)
    y_leastsquares = np.polyval(coeff, x_lin)
    plt.scatter(x, y, label=r"data $(x_i, y_i)$")
    plt.plot(x_lin, f(x_lin), color="black", label=r"function $f(x)$")
    plt.plot(x_lin, y_leastsquares, color="red", label=f"least squares polynomial ($m = {m}$)")
    plt.legend()
    plt.show()

# We notice that when the degree increases, the precision of the interpolation
# decreases. In fact, the least squares polynomial of degree m=7 gives in this
# case a better approximation than the one of degree m=15.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 3:** Evaluate the approximation error

$$
E(p_m^{LS},f)=\max_{x \in [0,10]} \vert f(x) - p_m^{LS}(x) \vert
$$

for $m=1,\ldots,15$. (Do this on a very fine grid.) Then plot the error in terms of $m$ in semi-logarithmic scale. What behavior do you observe for high values of $m$? For which value of $m$ is the approximation error the smallest?
</div>

In [ ]:
### BEGIN SOLUTION
errors = []
m_list = np.arange(1, 16)
for m in m_list:
    coeff = np.polyfit(x, y, m)
    x_lin_fine = np.linspace(0, 10, 10000)
    y_leastsquares = np.polyval(coeff, x_lin_fine)
    y_true = f(x_lin_fine)
    errors.append(np.max(np.abs(y_true - y_leastsquares)))

plt.semilogy(m_list, errors)
plt.xlabel(r"$m$")
plt.ylabel(r"approximation error")
plt.grid(True, which="major",ls="-")
plt.grid(True, which="minor",ls="--")
plt.ylim(0.09, 1.5)
plt.show()

# We notice that the polynomial of degree m=7 gives a better approximation than
# m=4. However, the polynomial of degree m=15 gives a bad approximation at the
# extremities of the interval. This is due to the fact that the least squares
# approximation becomes more and more unstable (for equally distributed nodes)
# when m approaches n.

# We can notice that the approximation error decreases up to the value m=6, 
# but afterwards the error increases as the number of data points (n=19) is
# not big enough.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 4:** For $m=4$, estimate the variance of the noise using the formula 

$$
\hat{\sigma}^2=\frac{1}{n-m}\sum_{i=1}^{n+1}\left(y_i-p_m^{LS}(x_i)\right)^2
$$

whose description you can find in the lecture notes on page 53. Check that in the previous exercise, the smallest value of $E(p_m^{LS},f)$ for any $m$ is of the order of $\hat{\sigma}\sqrt{\frac{m+1}{n+1}}$. Also,
compare the smallest error with $\sigma\sqrt{\frac{m+1}{n+1}}$. Remember that $n+1 = 20$ was the number of data points and $\sigma = 0.1$ the standard deviation of the noise we added to the data.
</div>

In [ ]:
### BEGIN SOLUTION
n = len(x) - 1
m = 4
coeff = np.polyfit(x, y, m)
y_leastsquares = np.polyval(coeff, x)

sigma2_est = np.sum((y - y_leastsquares)**2) / (n - m)
print("estimated variance of noise: σ² = {:.4f}".format(sigma2_est))

min_error_est = np.sqrt(sigma2_est) * np.sqrt((m + 1) / (n + 1))
print("estimated smallest error: σ √(m+1)/(n+1) = {:.4f}".format(min_error_est))

sigma = 0.1
min_error_eff = sigma * np.sqrt((m + 1) / (n + 1))
print("theoretical smallest error: σ √(m+1)/(n+1) = {:.4f}".format(min_error_eff))

# In the presence measurement errors, it is usually not possible to achieve
# zero error with a least squares polynomial. If the errors are iid random
# variables with expected value zero and variance σ² > 0, one can prove that
# the error E(p_m, f) is of the order of σ √(m+1)/(n+1). The computed estimate
# for this error quite accurately reflects the actually observed smallest error
# in the above plot.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 5:** Repeat all of the above exercises with the data contained in the file `big_data.txt`, which includes $n+1=20000$ data points following the same model as the data in `data.txt`. Compare for which $m$ the smallest approximation error is obtained for the two data sets and how big it is respectively.
</div>

In [ ]:
### BEGIN SOLUTION
def f(x):
    return np.sin(x) + x

# Plot data
data = np.loadtxt("big_data.txt")
x = data[:, 0]
y = data[:, 1]
x_lin = np.linspace(0, 10, 100)
plt.scatter(x, y, label=r"data $(x_i, y_i)$")
plt.plot(x_lin, f(x_lin), color="black", label=r"function $f(x)$")
plt.legend()
plt.show()

# Plot least squares approximants
m_list = [4, 7, 15]
for m in m_list:
    coeff = np.polyfit(x, y, m)
    y_leastsquares = np.polyval(coeff, x_lin)
    plt.scatter(x, y, label=r"data $(x_i, y_i)$")
    plt.plot(x_lin, f(x_lin), color="black", label=r"function $f(x)$")
    plt.plot(x_lin, y_leastsquares, color="red", label=f"least squares polynomial ($m = {m}$)")
    plt.legend()
    plt.show()

# Plot errors
errors = []
m_list = np.arange(1, 16)
for m in m_list:
    coeff = np.polyfit(x, y, m)
    x_lin_fine = np.linspace(0, 10, 1000)
    y_leastsquares = np.polyval(coeff, x_lin_fine)
    y_true = f(x_lin_fine)
    errors.append(np.max(np.abs(y_true - y_leastsquares)))

plt.semilogy(m_list, errors)
plt.xlabel(r"$m$")
plt.ylabel(r"interpolation error")
plt.grid(True, which="major",ls="-")
plt.grid(True, which="minor",ls="--")
plt.show()

# Compute variance/noise estimators
n = len(x) - 1
m = 4
coeff = np.polyfit(x, y, m)
y_leastsquares = np.polyval(coeff, x)

sigma2_est = np.sum((y - y_leastsquares)**2) / (n - m)
print("estimated variance of noise: σ² = {:.4f}".format(sigma2_est))

min_error_est = np.sqrt(sigma2_est) * np.sqrt((m + 1) / (n + 1))
print("estimated smallest error: σ √(m+1)/(n+1) = {:.4f}".format(min_error_est))

sigma = 0.1
min_error_eff = sigma * np.sqrt((m + 1) / (n + 1))
print("theoretical smallest error: σ √(m+1)/(n+1) = {:.4f}".format(min_error_eff))

# With n+1=20000 nodes, the least squares polynomials of degree m ≤ 15 are
# accurate since the number of data is very high. Therefore, the approximation
# error decreases almost up to the value m = 11.

# The minimal approximation error is obtained for m = 11 and is around 0.006
# which is approximately the same as the estimate σ √(m+1)/(n+1).
### END SOLUTION

<hr style="clear:both">

### Mysterious finite differences formula

<div class="alert alert-success">
    
**Exercise 6 (Theoretical):** Consider the general finite differences formula

$$
D_hf({x})=\frac{\frac{1}{3} f({x}+h)+\beta\, f({x})- (\frac{1}{2}+\beta)\,f(x-h)+
\frac{1}{6}f(x-2h)}{h}
$$

to approximate $f'(x)$ with $\beta\, \in \mathbb{R}$ independent of $h$. If we suppose that the function $f$ is regular enough, which of the following statements is true? [Only one answer is possible]

1. If $\beta = \frac{1}{2}$, the formula $D_h f$ is of second order but not of third order.

2. If $\beta \neq \frac{1}{2}$, the formula does not approximate $f'(x)$ when $h\rightarrow 0$.

3. If $\beta = -\frac{5}{6}$, the formula $D_h f$ is of first order.

4. The formula $D_h f$ converges to $f'(x)$ for all $\beta \in \mathbb{R}$ when $h\rightarrow 0$.

5. If $\beta = -\frac{1}{6}$, the formula $D_h f$ is of first order.

*Hint:* Insert the Taylor series expansions of $f(x + h)$, $f(x - h)$, and $f(x - 2h)$ into the formula and group all terms of the same derivative order ($f(x),f'(x),f''(x), \dots$) together to see how it behaves depending on $\beta$ and $h$.
</div>

=== BEGIN MARK SCHEME ===

The answer 2 is correct. Expanding $f(x + h)$, $f(x - h)$, and $f(x - 2h)$ in Taylor series of first order around $x$, we get

\begin{align*}
D_h f(x) = \frac{1}{h} 
	\Bigg[&\frac{1}{3} \bigg(f(x) + hf'(x) + \mathcal{O}(h^2) \bigg) \\
           &+ \beta f(x) \\
           &-\left( \frac{1}{2} + \beta \right) \bigg( f(x) - h f'(x) + \mathcal{O}(h^2)  \bigg) \\
       &\frac{1}{6} \bigg( f(x) - 2 h f'(x) +  \mathcal{O}(h^2)  \bigg)
	\Bigg].
\end{align*}

Regrouping the terms yields 

\begin{align*}
D_h f(x) &= f(x) \frac{1}{h} \underbrace{\bigg( \frac{1}{3} + \beta - \frac{1}{2} - \beta + \frac{1}{6} \bigg)}_{=0} + f'(x) \underbrace{\bigg( \frac{1}{3} + \frac{1}{2} + \beta - \frac{1}{3} \bigg)}_{= \frac{1}{2} + \beta} + \mathcal{O}(h) \\
&= f'(x)\bigg(\frac{1}{2} + \beta\bigg) + \mathcal{O}(h).
\end{align*}

Therefore, unless $\beta = \frac{1}{2}$, $D_h f(x)$ will not approximate $f'(x)$ when $h \to 0$.

=== END MARK SCHEME ===

<hr style="clear:both">

### Forward and central finite differences

<div class="alert alert-success">
    
**Exercise 7:** Implement the functions `delta_h_forward` and `delta_h_central` which, given a function $f$, a point $\overline{x}$, and a value $h$, compute the forward finite differences approximation 
\begin{equation}
\delta_h^{+}f(\overline{x}) = \frac{f(\overline{x} + h) - f(\overline{x})}{h}
\end{equation}
and the central finite difference approximation 
\begin{equation}
\delta_h^{c}f(\overline{x}) = \frac{f(\overline{x} + h) - f(\overline{x} - h)}{2h}
\end{equation}
respectively.
</div>

In [ ]:
def delta_h_forward(f, x_bar, h):
    ### BEGIN SOLUTION
    return (f(x_bar + h) - f(x_bar)) / h
    ### END SOLUTION

def delta_h_central(f, x_bar, h):
    ### BEGIN SOLUTION
    return (f(x_bar + h) - f(x_bar - h)) / (2 * h)
    ### END SOLUTION

Let us run a few simple checks on your implementation, to see if there are no issues.

In [ ]:
assert np.isclose(D := delta_h_forward(lambda x: x, 0, 0.1), 1), f"'delta_h_forward(f, 0, 0.1)' for 'f(x) = x' should return 'f'(0) ≈ 1', but got {D}";assert np.isclose(D := delta_h_central(lambda x: x, 0, 0.1), 1), f"'delta_h_central(f, 0, 0.1)' for 'f(x) = x' should return 'f'(0) ≈ 1', but got {D}";assert np.isclose(D := delta_h_forward(lambda x: x**2, 0.5, 1e-10), 1), f"'delta_h_forward(f, 0, 0.1)' for 'f(x) = x^2' should return 'f'(1/2) ≈ 1', but got {D}";assert np.isclose(D := delta_h_central(lambda x: x**2, 0.5, 1e-10), 1), f"'delta_h_central(f, 0, 0.1)' for 'f(x) = x^2' should return 'f'(1/2) ≈ 1', but got {D}";print("Nice! Your function worked well on the simple examples.")

We now consider the function

\begin{equation}
f(x)=x\log(x)-\sin^2(x)
\end{equation}

for $x>0$ for which we want to approximate the first derivative in
$\bar{x}=1.$

<div class="alert alert-success">

**Exercise 8:** Approximate the derivative of $f$ using the two methods you have implemented in the previous exercises for $h = 0.1 \cdot (\frac{1}{2})^{i}$ for $i=0,1,2,3,4,5$, and compute for each $h$ the error $\varepsilon_h=\vert f^{'}(\bar{x})-\delta_h  (\bar{x})\vert$, where $\delta_h$ is either the forward finite differences or the central finite differences operator. Plot the errors $\varepsilon_h$ in terms of $h$ on a graph in logarithmic scale. Comment on the obtained results.

*Hint:* Use `plt.loglog` and compare the graphs of $\varepsilon_h$ with the graphs defined by the curves $(h,h)$ and $(h,h^2)$.
</div>

In [ ]:
def f(x):
    ### BEGIN SOLUTION
    return x * np.log(x) - np.sin(x) ** 2
    ### END SOLUTION

def df(x):
    ### BEGIN SOLUTION
    return 1 + np.log(x) - 2 * np.sin(x) * np.cos(x)
    ### END SOLUTION

### BEGIN SOLUTION
x_bar = 1
errors_forward = []
errors_central = []

h_list = 0.1 * 0.5 ** np.arange(6)
for h in h_list:
    errors_forward.append(abs(df(x_bar) - delta_h_forward(f, x_bar, h)))
    errors_central.append(abs(df(x_bar) - delta_h_central(f, x_bar, h)))

plt.loglog(h_list, errors_forward, label=r"$\varepsilon_h$ forward")
plt.loglog(h_list, errors_central, label=r"$\varepsilon_h$ central")
plt.loglog(h_list, h_list, linestyle="--", c="tab:blue", label=r"$(h, h)$")
plt.loglog(h_list, h_list ** 2, linestyle="--", c="tab:orange", label=r"$(h, h^2)$")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

# We notice that for the forward finite differences method, the error decreases
# with a slope (h, h) in logarithmic scale, showing that the method is of first
# order. For the central finite differences method, the error decreases with
# slope (h, h^2), which means it is of second order.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 9:** Repeat the previous exercise for $i = 0, 1,\ldots, 30$. Comment the results.
</div>

In [ ]:
### BEGIN SOLUTION
x_bar = 1
errors_forward = []
errors_central = []

h_list = 0.1 * 0.5 ** np.arange(31)
for h in h_list:
    errors_forward.append(abs(df(x_bar) - delta_h_forward(f, x_bar, h)))
    errors_central.append(abs(df(x_bar) - delta_h_central(f, x_bar, h)))

plt.loglog(h_list, errors_forward, label=r"$\varepsilon_h$ forward")
plt.loglog(h_list, errors_central, label=r"$\varepsilon_h$ central")
plt.loglog(h_list, h_list, linestyle="--", c="tab:blue", label=r"$(h, h)$")
plt.loglog(h_list, h_list ** 2, linestyle="--", c="tab:orange", label=r"$(h, h^2)$")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

# We notice that for the forward finite differences method, the errors decreases
# initially with slope (h, h) in logarithmic scale and then, for h < √1e-16 = 1e-8,
# it increases with slope (h, 1/h). This is due to round-off errors. Similarly,
# for the central finite differences method, the error decreases initially with
# slope (h, h^2) and then, for h < ∛1e-16 ≈ 1e-5, it increases with slope (h, 1/h)
# because of the round-off errors.
### END SOLUTION

<hr style="clear:both">

## The end

Amazing! You have reached the end of the sixth exercise notebook. We wish you a relaxing study break!